<a href="https://colab.research.google.com/github/lokeshtalamala1/Deep_Learning_Assignment_1/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#initializing my project and entity in wandb using its atributes
#Deep_Learning_Assignment_1
#cs24m023-indian-institute-of-technology-madras

!pip install wandb tensorflow
import wandb
wandb.init(project='Deep_Learning_Assignment_1', entity='cs24m023-indian-institute-of-technology-madras')


In [3]:
#import numpy ,fashion_mnist and load data

from keras.datasets import fashion_mnist
import numpy as np

((x_train,y_train),(x_test,y_test)) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
import matplotlib.pyplot as plt

In [5]:
#example image from each class from given data

#QUESTION 1

images = []
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for class_label in range(len(classes)):
    index = next(i for i, y in enumerate(y_train) if y == class_label)
    images.append(wandb.Image(x_train[index], caption=classes[class_label]))

wandb.log({"example image from each class": images})
plt.show()
wandb.finish()

In [6]:
#QUESTION 2 AND QUESTION 3
#creating a class with a layer as an object and parametes to the layer are :
    #input dimensions
    #activation_function
    #optimizer_function
    #weight initialization

#FORWARD PROPAGATE AND BACKWARD PROPAGATE
#feedforwardneuralnetwork for calculating all the layers over the model and loss functions are calculated
#with input parameters optimizer,activation function weight initialization
#number of epochs and size of each layer ,learnig rate theeta

import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# -------------------- FeedForwardNN Class --------------------
class FeedForwardNN:
    def __init__(self, input_size=None, hidden_nodes=None, output_size=None, layer_dims=None,
                 activation_hidden='relu', init_type='xavier'):
        """
        Initializes the neural network.

        You can specify the architecture in two ways:
          1. Directly pass a list of layer dimensions as 'layer_dims'.
          2. Or pass the input size, list of hidden layer node counts, and output size.
        """
        if layer_dims is not None:
            self.layer_dims = layer_dims
        else:
            if input_size is None or hidden_nodes is None or output_size is None:
                raise ValueError("Either layer_dims or input_size, hidden_nodes, and output_size must be provided")
            self.layer_dims = [input_size] + hidden_nodes + [output_size]

        self.L = len(self.layer_dims) - 1  # Number of layers excluding the input layer
        self.init_type = init_type
        self.activation_hidden = activation_hidden
        self.parameters = self.initialize_parameters(self.layer_dims, init_type)

    # ---- Activation Functions and Their Gradients ----
    def sigmoid(z):
        return 1.0 / (1.0 + np.exp(-z))

    def dsigmoid(z):
        s = FeedForwardNN.sigmoid(z)
        return s * (1 - s)

    def tanh(z):
        return np.tanh(z)

    def dtanh(z):
        return 1 - np.tanh(z)**2

    def relu(z):
        return np.maximum(0, z)

    def drelu(z):
        return np.array(z > 0, dtype=float)

    def softmax(z):
        z_exp = np.exp(z - np.max(z, axis=0, keepdims=True))
        return z_exp / np.sum(z_exp, axis=0, keepdims=True)

    def dsoftmax(z):
        s = FeedForwardNN.softmax(z.reshape(-1, 1))
        jacobian = np.diagflat(s) - np.dot(s, s.T)
        return jacobian

    def get_activation_function(self):
        if self.activation_hidden.lower() == 'sigmoid':
            return FeedForwardNN.sigmoid, FeedForwardNN.dsigmoid
        elif self.activation_hidden.lower() == 'tanh':
            return FeedForwardNN.tanh, FeedForwardNN.dtanh
        elif self.activation_hidden.lower() == 'relu':
            return FeedForwardNN.relu, FeedForwardNN.drelu
        else:
            raise ValueError("Unsupported activation function. Choose from 'sigmoid', 'tanh', or 'relu'.")

